# Nodo

In [1]:
"""
Definicion de la clase nodo, servira para crear objetos de tipo nodo con los atributos etiqueta,dirigido y aristas

"""


class Nodo:
    def __init__(self, etiqueta,dist=int(0), dirigido=False):  # Constructor de la clase
        self.__etiqueta = etiqueta  # identificador del nodo
        self.__dirigido = dirigido  # parametro dirigido se usa para saber si el grafo es dirigido
        self.__aristas = set()  # Estructura de datos tipo set, util para mantener las aristas conectadas al nodo
        self.__dist    = (dist)

    def __str__(self):  # Sobrescritura del metodo str
        cadena_retorno = ""
        for arista in self.__aristas:
            cadena_retorno += str(arista)

        return cadena_retorno

    def get_aristas(self):  # Getter de la arista regresa una arista especifica en el nodo
        return self.__aristas

    def get_etiqueta(self):  # Getter de la etiqueta regresa la etiqueta que identifica al nodo
        return self.__etiqueta

    # Getter de las aristas salientes, si el grafo es dirigido regresa las aristas que salen del nodo
    def get_aristas_salientes(self):
        if not self.__dirigido:
            return self.__aristas

        aristas_salientes = []
        for arista in self.__aristas:
            if arista.get_nodo_fuente() == self:
                aristas_salientes.append(arista)

        return aristas_salientes

    # Getter de las aristas entrantes, si el grafo es dirigido regresa las aristas que entran al nodo
    def get_aristas_entrantes(self):
        if not self.__dirigido:
            return self.__aristas

        aristas_entrantes = []
        for arista in self.__aristas:
            if arista.get_nodo_destino() == self:
                aristas_entrantes.append(arista)

        return aristas_entrantes

    # Metodo para agregar una arista
    def add_arista(self, arista):
        self.__aristas.add(arista)  # se agrega una arista al set que guarda las aristas del nodo

    def remove_arista(self, arista):
        if arista in self.__aristas:  # Si se encuentra la arista en el nodo se remueve la arista
            self.__aristas.remove(arista)  # se remueve una arista del set que guarda las aristas
        else:  # si no se encuentra la arista en el nodo lanza el sigueinte error
            raise ValueError(
                "No se pudo encontrar la arista {0} en el  nodo {1}".format(str(arista), str(self)))

    def set_aristas(self, aristas):  # Setter de las aristas
        self.__aristas = aristas
    def set_distancia(self,dist):
        self.__dist=dist
    def get_distancia(self):
        return self.__dist
    def set_etiqueta(self,etiqueta):
        self.__etiqueta=etiqueta

    def __str__(self):
        return self.__etiqueta



# Arista

In [2]:
"""
Definicion de la clase arista, servira para crear un objeto de tipo arista con los atributos nodo de inicio y nodo final
Ademas del parametro dirigido y el peso
"""


class Arista:
    # Constructor de la clase arista
    def __init__(self, nodo_inicio, nodo_final,peso=1, dirigido=False):
        self.__nodo_inicio = nodo_inicio  # nodo fuente
        self.__nodo_final = nodo_final  # nodo destino
        self.__peso = peso
        self.__dirigido = dirigido  # parametro para seber si el grafo es dirigido

    # Sobreescritura del metodo str
    def __str__(self):
        if self.__dirigido:
            print_pattern = "{0}->{2}->{1}"
        else:
            print_pattern = "{0} -{2}- {1} "
        return print_pattern.format(self.__nodo_inicio.get_etiqueta(), self.__nodo_final.get_etiqueta(),self.__peso)

    # Getter del nodo fuente, regresa el nodo fuente de la aristas
    def get_nodo_fuente(self):
        return self.__nodo_inicio

    # Getter del nodo destino, regresa el nodo destino  de la aristas
    def get_nodo_destino(self):
        return self.__nodo_final
    # Getter del peso
    def get_peso(self):
        return self.__peso
    # Sobrescritura del metodo less than
    def __lt__(self, other):
        return self.__peso < other.get_peso()

    # Sobrecritura de los metodos hash y eq
    def __eq__(self, otro):
        nodo_inicio_igual = self.__nodo_inicio == otro.get_nodo_fuente()
        nodo_final_igual = self.__nodo_final == otro.get_nodo_destino()

        return nodo_inicio_igual == nodo_final_igual == True

    def __hash__(self):

            #print(type(self.get_nodo_destino()))

            return hash(self.get_nodo_fuente().get_etiqueta() +
                    self.get_nodo_destino().get_etiqueta() + str(self.__peso))


# Grafo

In [3]:
"""
definición de la clase grafo, posee un diccionario para almacenar objetos de tipo nodo
las aristas se guardan en una estructura tipo set

"""
import random
import itertools
import math
import ctypes
import collections
from collections import defaultdict
from collections import deque
import heapq
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

class Grafo:
    def __init__(self, dirigido=False):
        self.nodos = {}  # diccionario para almacenar los nodos con su etiqueta como llave
        self.__aristas = set()  # mantiene las aristas guardadas en un conjunto
        self.__dirigido = dirigido  # parametro para saber si el grafo es dirigido

    def __str__(self):  # sobreescritura del metodo string
        imp_cadena = ""
        nodos_copy=self.nodos
        for llave in nodos_copy.copy():
            if str(nodos_copy[llave])=="":
               # nodos_copy.pop(llave)
                pass
            else:
                new_string = str(nodos_copy[llave]).replace(llave, "")
                if  not new_string.startswith(llave):
                    if new_string.startswith(" --"):
                        imp_cadena += llave + new_string + '\n'
                    else:
                        if new_string.endswith("-- "):
                            t = new_string.rsplit('--', 1)
                            u = ''.join(t)
                        else:
                            t = new_string.rsplit('--',1)
                            u = ''.join(t)
                        if u=='':
                            u=''
                        imp_cadena += "{0} -- ".format(llave) + u + '\n'



        return imp_cadena

    def add_nodo(self, etiqueta,distancia=0):  # metodo para añadir nodos al grafo
        if etiqueta not in self.nodos:
            self.nodos[etiqueta] = Nodo(etiqueta,distancia, self.__dirigido)

    def add_arista(self, etiqueta_inicio, etiqueta_final,peso=1):  # metodo para añadir aristas al grafo
        nodo_fuente = self.get_nodo(etiqueta_inicio)
        nodo_final = self.get_nodo(etiqueta_final)
        mirror_edge = None
        if (nodo_fuente or nodo_final) is None:  # si no existe el nodo fuente o el nodo destino salta el error
            #print(type(nodo_fuente),type(nodo_final))
            raise ValueError("No se puede encontrar el nodo fuente  o el nodo destino ")
        if self.__dirigido:  # Si el grafo es dirigido se agrega la arista tal cual
            arista = Arista(nodo_fuente, nodo_final,peso, self.__dirigido)
        else:
            for aristaen in self.get_aristas():
                nodo_fuente_arista = aristaen.get_nodo_fuente()
                nodo_destino_arista = aristaen.get_nodo_destino()

                if (nodo_destino_arista == nodo_fuente) and (nodo_fuente_arista == nodo_final):
                    mirror_edge = True
            if not mirror_edge:
                arista = Arista(nodo_fuente, nodo_final,peso, self.__dirigido)
            else:
                #print(type(nodo_fuente))
                #print(type(nodo_final))
                arista = Arista(nodo_final, nodo_fuente,peso, self.__dirigido)

        nodo_fuente.add_arista(arista)

        if nodo_fuente != nodo_final:
            nodo_final.add_arista(arista)

        self.__aristas.add(arista)

    def remove_arista(self, etiqueta_inicio, etiqueta_final):  # metodo para eliminar aristas en el grafo
        nodo_fuente = self.get_nodo(etiqueta_inicio)
        nodo_destino = self.get_nodo(etiqueta_final)

        if (nodo_fuente or nodo_destino) is None:
            raise ValueError("No se encontro el nodo fuente o el nodo destino en el grafo")

        arista = Arista(nodo_fuente, nodo_destino, self.__dirigido)

        if arista not in self.__aristas:
            raise ValueError(
                "no se encuentra la arista  {0} en el grafo".format(str(Arista)))

        nodo_fuente.remove_arista(arista)
        nodo_destino.remove_arista(arista)
        self.__aristas.remove(arista)

    def remove_nodo(self, etiqueta_nodo):  # metodo para eliminar nodos en el grafo
        if etiqueta_nodo not in self.nodos:
            raise ValueError(
                "No se encontro el nodo {0} en el grafo".format(etiqueta_nodo))

        nodo = self.nodos[etiqueta_nodo]

        copia_aristas_entrantes = nodo.get_aristas_entrantes().copy()  # Removemos la entrada de las aristas entrantes
        for arista in copia_aristas_entrantes:
            nodo_adyacente = arista.get_nodo_fuente()
            nodo_adyacente.remove_arista(arista)

            if arista in self.__aristas:
                self.__aristas.remove(arista)

        copia_aristas_salientes = nodo.get_aristas_salientes().copy()
        for arista in copia_aristas_salientes:  # Removemos la entrada de las aristas salientes
            adjacent_vertex = arista.get_nodo_destino()
            adjacent_vertex.remove_arista(arista)

            if arista in self.__aristas:
                self.__aristas.remove(arista)

        self.nodos.pop(etiqueta_nodo)

        nodos = self.get_nodos()
        for nodo in nodos:
            aristas = nodos[nodo].get_aristas()
            copia_aristas = aristas.copy()
            for arista in aristas:
                nodo = arista.get_nodo_fuente()
                if nodo.get_etiqueta() == etiqueta_nodo:
                    copia_aristas.remove(arista)
                nodo.set_aristas(copia_aristas)

    # empieza la definicion de getters
    def get_nodo(self, etiqueta):
        return self.nodos.get(etiqueta)

    def get_nodos(self): #metodo para obtener los nodos
        return self.nodos

    def get_aristas(self): # Metodo para obtener aristas
        return self.__aristas

    def get_grado(self, etiqueta):  # Metodo para obtener el grado
        if self.es_dirigido():
            grado = len(self.get_nodo(etiqueta).get_aristas_entrantes()) + \
                    len(self.get_nodo(etiqueta).get_aristas_salientes())
        else:
            grado = len(self.get_nodo(etiqueta).get_aristas_salientes())
        return grado

    def es_dirigido(self):  # Metodo que regresa si el grafo es dirigido
        return self.__dirigido

    # comienza la definicion de los metodos para generar grafos aleatorios
    def grafo_malla(self, m, n):
        for i in range(m * n):  # se comienza creando los n nodos
            self.add_nodo(str(i))
        for j in range(m):
            # print("j:"+str(j))
            index_horizontal = j * n
            # print("index:"+str(index_horizontal))
            for i in range(index_horizontal, index_horizontal + n):
                # print("esa es:"+str(i))
                # if(i<10):
                if (i != (n - 1) + index_horizontal):
                    # print("agregando arista {0},{1}".format(i,(i+1)))
                    self.add_arista(str(i), str(i + 1),random.randrange(1,1000))
                if j != (m - 1):
                    # print("agregando arista {0},{1}".format(i, (i + n)))
                    self.add_arista(str(i), str(i + n),random.randrange(1,1000))
        return self

    def grafo_erdos_renyi(self, n, m, dirigido=False, auto=False):  # metodo para generar modelo de ErdosRenyi
        self.__dirigido = dirigido
        for i in range(n):
            self.add_nodo(str(i))
        while len(self.get_aristas()) != m:
            n1 = (random.randrange(n))
            n2 = (random.randrange(n))
            if not auto:
                if n1 != n2:
                    self.add_arista(str(n1), str(n2),random.randrange(1,1000))
            else:
                self.add_arista(str(n1), str(n2),random.randrange(1,1000))
        #print(len(self.get_aristas()))
        return self

    def grafo_gilbert(self, n, p, dirigido=False, auto=False):
        self.__dirigido = dirigido
        for i in range(n):
            self.add_nodo(str(i))
        for i in range(n):
            for j in range(n):
                if not auto:
                    if (i != j):
                        if random.random() <= p:
                            # print("creando arista ({0},{1})".format(i,j))
                            self.add_arista(str(i), str(j),random.randrange(1,1000))
        #print(len(self.get_aristas()))
        return self

    def dorogovtsev_mendes(self, n, dirigido=False):
        self.__dirigido = dirigido
        for i in range(3):
            #print(i)
            self.add_nodo(str(i))
        self.add_arista("0", "1",random.randrange(1,1000))
        self.add_arista("0", "2",random.randrange(1,1000))
        self.add_arista("1", "2",random.randrange(1,1000))
        # print(self.get_grado("0"))
        while len(self.get_nodos()) != n:
            nodo_nuevo = str(len(self.get_nodos()) )
            print(nodo_nuevo)
            self.add_nodo(nodo_nuevo)
            arista_random = random.choice(list(self.get_aristas()))
            nodo_fuente = arista_random.get_nodo_fuente()
            etiqueta_nodo_fuente = nodo_fuente.get_etiqueta()
            nodo_destino = arista_random.get_nodo_destino()
            etiqueta_nodo_destino = nodo_destino.get_etiqueta()
            self.add_arista(nodo_nuevo, etiqueta_nodo_fuente,random.randrange(1,1000))
            self.add_arista(nodo_nuevo, etiqueta_nodo_destino,random.randrange(1,1000))
        #print("numero de nodos: {}".format(len(self.get_nodos())))
        #print("numero de aristas: {}".format(len(self.get_aristas())))
        return self

    def grafo_barabasi_albert(self, n, d, dirigido=False, auto=False):
        self.__dirigido = dirigido
        for i in range(n):  # se generan n nodos
            self.add_nodo(str(i))
        for i in range(n):  # iteramos entre todos los posibles pares
            for j in range(n):
                if not auto:  # si no se permiten autociclos
                    if i != j:
                        # print("El grado del nodo {0} es {1}".format(j, self.get_grado(str(j))))
                        # probamos en base a la probabilidad que depende del grado del nodo
                        if (self.get_grado(str(j)) < d) and (self.get_grado(str(i)) < d):
                            p = 1 - (self.get_grado(str(j)) / d)
                            if random.random() <= p:  # si el numero random es menor o igual a la probailidad se crea
                                # print("creando arista ({0},{1})".format(i, j))
                                self.add_arista(str(i), str(j),random.randrange(1,1000))
                else:  # lo  mismo pero si se permiten autociclos
                    if (self.get_grado(str(j)) < d) and (self.get_grado(str(i)) < d):
                        p = 1 - (self.get_grado(str(j)) / d)
                        if random.random() <= p:
                            # print("creando arista ({0},{1})".format(i, j))
                            self.add_arista(str(i), str(j),random.randrange(1,1000))
        #print(len(self.get_aristas()))
        return self

    def grafo_geografico(self, n, r, dirigido=False, auto=False):  # metodo para generar el modelo geografico
        self.__dirigido = dirigido  # parametro dirigido
        for i in range(n):  # se crean n nodos
            self.add_nodo(str(i))
        posicion_nodos = {}  # diccionario para mantener las cordenadas de los nodos
        for nodo in self.get_nodos():  # asignamos cordenadas  a los nodos
            llave = nodo
            posicion_random = (round(random.random(),3), round(random.random(),3))
            posicion_nodos.update({llave: posicion_random})
        combinaciones = itertools.combinations(posicion_nodos, 2)  # todos los posibles pares si el grafo es no dirigido
        permutaciones = itertools.permutations(posicion_nodos, 2)  # todos los posibles pares si el grafi es dirigido
        if not dirigido:  # si el grafo es no dirigido usamos las combinaciones para comparar la distancia de los pares
            for combinacion in combinaciones:
               # print(combinacion)
                nodo_fuente = combinacion[0]
                nodo_destino = combinacion[1]
                cordenadas_nodo_fuente = posicion_nodos.get(combinacion[0])
                cordenadas_nodo_destino = posicion_nodos.get(combinacion[1])
                nodo_fuente_x = cordenadas_nodo_fuente[0]
                nodo_fuente_y = cordenadas_nodo_fuente[1]
                nodo_destino_x = cordenadas_nodo_destino[0]
                nodo_destino_y = cordenadas_nodo_destino[1]
                # calculamos la distancia entre pares
                distancia = math.sqrt((nodo_destino_x - nodo_fuente_x) ** 2 + (nodo_destino_y - nodo_fuente_y) ** 2)
                # print(distancia)
                # si la distancia es menor o igual a r se genera la arista ente pares
                if (distancia <= r):
                    self.add_arista(nodo_fuente, nodo_destino,random.randrange(1,1000))
        else:  # lo mismo pero cuando el grafo es no dirigido usamos permutaciones
            for permutacion in permutaciones:
               # print(permutacion)
                nodo_fuente = permutacion[0]
                nodo_destino = permutacion[1]
                cordenadas_nodo_fuente = posicion_nodos.get(permutacion[0])
                cordenadas_nodo_destino = posicion_nodos.get(permutacion[1])
                nodo_fuente_x = cordenadas_nodo_fuente[0]
                nodo_fuente_y = cordenadas_nodo_fuente[1]
                nodo_destino_x = cordenadas_nodo_destino[0]
                nodo_destino_y = cordenadas_nodo_destino[1]
                distancia = math.sqrt((nodo_destino_x - nodo_fuente_x) ** 2 + (nodo_destino_y - nodo_fuente_y) ** 2)
                # print(distancia)
                if (distancia <= r):
                    self.add_arista(nodo_fuente, nodo_destino,random.randrange(1,1000))
        return self
    def get_nodos_adyacentes(self,node): #metodo para obtener nodos adyacentes
        nodos_adyacentes=[]
        nodo=self.get_nodo(node)
        for arista in nodo.get_aristas():
            if arista.get_nodo_fuente()!=nodo:
                nodos_adyacentes.append(arista.get_nodo_fuente().get_etiqueta())
            else:
                nodos_adyacentes.append(arista.get_nodo_destino().get_etiqueta())
        return nodos_adyacentes
    def get_hijos(self,node): #metodo para obtener hijos dado un nodo
        nodos_adyacentes=[]
        nodo=self.get_nodo(node)
        for arista in nodo.get_aristas():
            if arista.get_nodo_fuente()==nodo:
                nodos_adyacentes.append(arista.get_nodo_destino().get_etiqueta())
        return nodos_adyacentes
    def get_padres(self,node): #metodo para obtener los ancestros de un nodo
        nodos_adyacentes=[]
        nodo=self.get_nodo(node)
        for arista in nodo.get_aristas():
            if arista.get_nodo_fuente()!=nodo:
                nodos_adyacentes.append(arista.get_nodo_fuente().get_etiqueta())
        return nodos_adyacentes

    def get_peso_arista(self, u, v):
        nodo=self.get_nodo(u)
        aristas= nodo.get_aristas()
        for arista in aristas:
            if (self.get_nodo(v).get_etiqueta()== arista.get_nodo_fuente().get_etiqueta()
                    or self.get_nodo(v).get_etiqueta()== arista.get_nodo_destino().get_etiqueta()):
                return  arista.get_peso()
    def bfs(self, s): # Metodo de algoritmo de busqueda bfs
        queue = deque([s]) # cola que nos permitira guadar los nodos para ejecutar el algoritmo bfs
        capa = {s: 0} # creamos la capa 0
        ancestro = {s: None} # diccionario que nos permitira guardar los nodos y la relacion con sus hijos
        arbol = Grafo() #creamos un nuevo objeto de tipo grafo para generar el arbol que nos da el algoritmo bfs
        while queue: #mientras haya elementos en la cola
            nodo = queue.popleft() #extraemos el nodo de la cola
            arbol.add_nodo(nodo) #agregamos el nodo al arbol
            for n in self.get_nodos_adyacentes(nodo):#agregamos los nodos adyacentes al nodo a su respectiva capa
                if n not in capa:
                    queue.append(n)
                    capa[n] = capa[nodo] + 1
                    ancestro[n] = nodo

        for key in ancestro: #se agregan todas las aristas al arbol
            if ancestro[key]!=None:
                arbol.add_arista(ancestro[key], key)

        return arbol

    def dfs_i(self, s): #Metodo de algoritmo de busqueda dfs iterativo
        arbol_dfs = Grafo() #creamos un nuevo objeto tipo grafo para almacenar el arbol creado por el algoritmo
        explorados = [s] # agregamos el nodo fuente a la lista de explorados
        #print(explorados)
        arbol_dfs.add_nodo(s)   #agregamos el nodo fuente al arbol
        u = self.get_nodo(s)
        stack = [] #inicializamos el stack
        while len(explorados)<len(self.get_nodos()):
            #print(stack)
            #print(explorados)
            #print("explorando:{}".format(u))
            aristas = u.get_aristas()
            for arista in aristas:   # añadir al stack todos los nodos adyacentes a u
                #print(arista)
                #print("etiqueta de U:{}".format(u.get_etiqueta()))
                #print("Nodo fuente:{}".format(arista.get_nodo_fuente()))
                #print("Nodo destino:{}".format(arista.get_nodo_destino()))
                if str(u.get_etiqueta()) == str(arista.get_nodo_fuente()): #verificamos la direccion de la arista
                    comp=True
                else:
                    comp=False
                v = arista.get_nodo_destino() if comp else arista.get_nodo_fuente()
                #print("u:{} y v:{}".format(u,v))
                if str(v) not in explorados: #se agrega al stack la arista del nodo explorado
                    stack.append((u.get_etiqueta(), v.get_etiqueta()))
            if not stack:
                break
            padre, hijo = stack.pop() # hacemos pop al stack
            if hijo not in explorados:
                arbol_dfs.add_nodo(hijo)
                #print(padre,hijo)
                arbol_dfs.add_arista(padre,hijo) # se agrega la arista nueva al arbol
                explorados.append(hijo) # se agrega el  nodo explorado a la lista de explorados

            u = self.get_nodo(hijo)
            #print("nuevo u:{}".format(u))

        return arbol_dfs



    def dfs_r(self, u): #Metodo de algoritmo de busqueda dfs recursivo

        arbol_dfs = Grafo()
        explorados = set()
        self.recursive_tool(u, arbol_dfs, explorados)

        return arbol_dfs

    def recursive_tool(self, u, arbol_dfs, explorados):

        arbol_dfs.add_nodo(u)
        explorados.add(u)
        u=self.get_nodo(u)
        aristas = u.get_aristas()

        for arista in aristas:
            v = arista.get_nodo_destino().get_etiqueta()
            if str(u.get_etiqueta()) == str(arista.get_nodo_fuente()):  # verificamos la direccion de la arista
                comp = True
            else:
                comp = False
            v = arista.get_nodo_destino().get_etiqueta() if comp else arista.get_nodo_fuente().get_etiqueta()
            if v in explorados:
                continue
            arbol_dfs.add_nodo(arista.get_nodo_fuente().get_etiqueta()) #se agegan los nodos correspondientes a la arista al grafo
            arbol_dfs.add_nodo(arista.get_nodo_destino().get_etiqueta())#se agegan los nodos correspondientes a la arista al grafo
            #print(arista.get_nodo_fuente().get_etiqueta(),arista.get_nodo_destino().get_etiqueta())
            arbol_dfs.add_arista(arista.get_nodo_fuente().get_etiqueta(),arista.get_nodo_destino().get_etiqueta())
            self.recursive_tool(v, arbol_dfs, explorados)

    def Dijkstra(self, s):
        arbol_caminos=Grafo() # se crea el objeto que contendra el arbol de caminos minimos
        self.__dist = dist = dict() #diccionario para almacenar distancias
        self.__prev = prev = dict() #diccionario util para almacenar los padres
        explorado= set()            # con esta estructura se matiene un seguimiento de los nodos ya explorados
        cola_prioridad = set()      # cola de prioridad

        dist[s] = 0 # hacemos la distancia de el nodo fuente 0
        prev[s] = s # agregamos a la estructura el nodo fuente
        cola_prioridad.add(s)   # agregamos el nodo fuente a la cola de prioridades

        while cola_prioridad:
            u = min(cola_prioridad, key=dist.get) #obtener el elemento con el valor minimo en la cola de prioridad
            for nodo in self.get_nodos_adyacentes(u): # para cada nodo en los nodos adyacentes al nodo u
                if nodo in explorado:
                    continue
                actualizar_distancia = self.get_distancia(u) + self.get_peso_arista(u, nodo) # actualizamos distancia
                if actualizar_distancia < self.get_distancia(nodo): #si es menor se almacena la nueva distancia
                    dist[nodo] = actualizar_distancia
                    prev[nodo] = u
                    cola_prioridad.add(nodo) # se agrega el nodo adyacente a u  a la cola de prioridad
            cola_prioridad.remove(u) # removemos u de la cola de prioridad
            explorado.add(u) # agregamos el nodo u a los nodos ya explorados
        nodos=self.get_nodos() # obtenemos los nodos del arbol original
        #print(self.get_nodos())
        arbol_caminos.add_nodo(s,0) # agregamos el nodo fuente al arbol de caminos
        caminos=dict() #estructura para mantener los caminos minimos desde el nodo fuente hacia cada uno de los nodos
        for nodo in nodos:
            arbol_caminos.add_nodo(nodo, self.get_distancia(nodo)) # agregamos el nodo al arbol de caminos minimos
            caminos[nodo]=self.get_camino(nodo) # utilizamos la funcion que nos ayuda a construir el camino y lo guardamos
        for camino in caminos.values(): # agregamos las aristas del arbol caminos minimos
            for index in range(0,len(camino)-1):
                peso = self.get_peso_arista(camino[index], camino[index+1])
                #print(camino)
                #print(camino[index])
                #print(camino[index+1])
                arbol_caminos.add_arista(camino[index],camino[index+1], peso)
        return  arbol_caminos

    def get_distancia(self, u): #funcion que ayuda al calculo de las distancias
       
        return self.__dist.get(u, math.inf)

    def get_camino(self, s): #funcion util para la constriccion del arbol de caminos

        path = [s]
        index=0
        while self.__prev[s] != s:
            s = self.__prev[s]
            path.append(s)

        return path[::-1]

    def KruskalD(self):
        MST = Grafo()  # Construimos el objeto tipo grafo para el arbol de expansion minima
        indexmap = []  # Creamos un mapa de indices para facilitar las cosas
        result = []  # This will store the resultant MST
        j = 0
        k = 0

        # Agregamos los nodos al arbol de expansion minima
        for node in self.get_nodos():
            MST.add_nodo(str(node))


        #Funcion para encontrar el conjunto al que pertenece el elemento x
        def pertenece_conjunto(padre, x):
            if padre[x] == x:
                return x
            return pertenece_conjunto(padre, padre[x])


        #Funcion para detecta ciclos en el grafo implementa el algoritmo de union by rank
        def union(padre, rank, x, y):
            raiz_de_x =pertenece_conjunto(padre, x)
            raiz_de_y =pertenece_conjunto(padre, y)
            if rank[raiz_de_x] < rank[raiz_de_y]:
                padre[raiz_de_x] = raiz_de_y
            elif rank[raiz_de_x] > rank[raiz_de_y]:
                padre[raiz_de_y] = raiz_de_x
            else:
                padre[raiz_de_y] = raiz_de_x
                rank[raiz_de_x] += 1
        # Agregamos nuestras aristas del grafo original en nuesto mapa de indices
        for arista in self.get_aristas():
            indexmap.append([int(arista.get_nodo_fuente().get_etiqueta()), int(arista.get_nodo_destino().get_etiqueta()),
                  self.get_peso_arista(arista.get_nodo_fuente().get_etiqueta(),
                                       arista.get_nodo_destino().get_etiqueta())])

        # Ordenamos las aristas
        indexmap = sorted(indexmap,
                            key=lambda item: item[2])

        padre = [] #Estructura para mantener los ancestros del en el arbol
        rank = []


        for node in range(len(self.get_nodos())):
            padre.append(node)
            rank.append(0) # agregamos el nodo 0 como el primer nodo padre


        while k < len(self.get_nodos())-1:

           #Tomamos la arista con el menor valor
            u, v, w = indexmap[j]
            j = j + 1
            x = pertenece_conjunto(padre, u)
            y = pertenece_conjunto(padre, v)

            # Si esto no causa un ciclo la agregamos al abol de expansion minima
            if x != y:
                k = k + 1
                MST.add_arista(str(u),str(v),w)
                union(padre, rank, x, y)


        costo_minimo= 0
        print("Aristas en el arbol de expansion minima")
        for arista in MST.get_aristas():
            peso=arista.get_peso()
            costo_minimo += peso
            print("{0} - {2} - {1} ".format((arista.get_nodo_fuente().get_etiqueta()),(arista.get_nodo_destino().get_etiqueta()), peso))
        mensaje="El costo del MST en KruskalD es: {}".format(costo_minimo)
        print(mensaje)
        
        
        #ctypes.windll.user32.MessageBoxW(0, mensaje, "MST KruskalD", 0x40)
        
        
        return MST


    def KruskalI(self):
        MST=Grafo() #Construimos el objeto tipo grafo para el arbol de expansion minima

        self.index_map = [0] * len(self.get_nodos()) #Definimos un mapa de indices para verificar aristas en los conjuntos
        self.aristas = []        # En esta estructura haremos una copia de las aristas de nuestro grafo original
        for i in range(len(self.get_nodos())): #inicializamos nuestro mapa de indices
            self.index_map [i] = []
        # Agregamos los nodos al arbol de expansion minima
        for node in self.get_nodos():
            MST.add_nodo(str(node))
        #funcion que nos ayuda a crear nuestro mapa de indices
        def Mapeo( u, v,p):
            self.index_map [u].append(v)
            self.index_map [v].append(u)
            self.aristas.append((p, (u, v)))
        #funcion util para la siguiente funcion que nos ayuda a encontrar el componente conectado
        def Visitado( v, visitado):
            visitado[v] = True
            for i in self.index_map [v]:
                if not visitado[i]:
                    Visitado(i, visitado)
        #Funcion para encontrar el componente conectado
        def conectado(self):
            explorado= [False] * len(self.get_nodos())
            Visitado(0, explorado)
            for j in range(1, len(self.get_nodos())):
                if not explorado[j]:
                    return False
            return True
        #Hacemos una copia de las aristas de nuestro grafo original para poder borrar las que no iran en el arbol
        for arista in self.get_aristas():
            Mapeo(int(arista.get_nodo_fuente().get_etiqueta()),int(arista.get_nodo_destino().get_etiqueta()),
                  self.get_peso_arista(arista.get_nodo_fuente().get_etiqueta(),
                                       arista.get_nodo_destino().get_etiqueta()))

        # Se ordenan las aristas
        self.aristas.sort(key=lambda a: a[0])
        peso_mst = 0

        print("Aristas en el arbol de expansion minima")

        #Iteramos sobre todos las aristas de acuerdo a como fueron odenadas
        for i in range(len(self.aristas) - 1, -1, -1):
            u = self.aristas[i][1][0]
            v = self.aristas[i][1][1]

            # Removemos las aristas que no iran en el arbol de expansion
            self.index_map [u].remove(v)
            self.index_map [v].remove(u)

            #verificamos si causa desconexion en el grafo para saber que aristas agregar al abol
            if conectado(self) == False:
                self.index_map [u].append(v)
                self.index_map [v].append(u)

                #Se imprimen las aristas que pertenecen al grafo se agregan al objeto tipo grafo que sera nuestro arbol
                print("{0} - {2} - {1}".format(u,v,self.get_peso_arista(str(u),str(v))))
                MST.add_arista(str(u),str(v),self.get_peso_arista(str(u),str(v)))
                peso_mst += self.aristas[i][0]
                mensaje="El costo del MST en KruskaLI es: {}".format(peso_mst)
        # Lanzamos una ventana de texto con el costo total de nuestro arbol de expansion minima
        print(mensaje)
        
        
        #ctypes.windll.user32.MessageBoxW(0,mensaje, "MST KruskalI", 0x40)
        
        
        return MST
    def Prim(self):
        s="0" # como lo que se implementa es basicamente dijkstra necesitamos un nodo fuente
        MST= Grafo() # Se crea el objeto grafo que guardara nuestro arbol de expansion minima
        self.ad_list= defaultdict(dict) #representacion en lista de adyacencia para facilirarnos la vida con hepq
        for nodo in self.nodos:
            for nodoadyacente in self.get_nodos_adyacentes(nodo):
                self.ad_list[nodo].update({nodoadyacente:self.get_peso_arista(nodo,nodoadyacente)})
        explorado= set([s]) # conjunto que guarda los nodos ya explorados
        aristas = [ (peso, s, nodo_destino)for nodo_destino, peso in self.ad_list[s].items()]
        heapq.heapify(aristas) # cola de prioridades
        mst_peso = 0
        # Agregamos los nodos al arbol de expansion minima
        for i in range(len(self.get_nodos())):
            MST.add_nodo(str(i))
        print("Aristas en el arbol de expansion minima")
        while aristas:
            peso, frm, nodo_destino = heapq.heappop(aristas) # sacamos el elemento de la cola de prioridades
            # si no se encuntra en los nodos ya explorados
            if nodo_destino not in explorado:
                mst_peso = mst_peso + peso # sumamos al peso del grafo
                explorado.add(nodo_destino) # agregamos a los nodos ya explorados
                print(" {0}- {2} -{1}".format(frm,nodo_destino,peso)) # imprimimos la arista para visualizacion
                MST.add_arista(str(frm),str(nodo_destino),int(peso)) # agregamos arista al objeto que mantiene el arbol
                for siguiente, peso in self.ad_list[nodo_destino].items():
                    if siguiente not in explorado:
                        heapq.heappush(aristas, (peso, nodo_destino, siguiente)) # hacemos push de los siguientes elementos
        mensaje = "El costo del MST en Prim's es: {}".format(mst_peso)
        print(mensaje)
        
        
        #ctypes.windll.user32.MessageBoxW(0, mensaje, "MST Prim's", 0x40)

        return MST



# Pasar de py a .dot

In [4]:
"""
Aqui definimos el metodo para guardar el grafo
Hacemos uso de la libreria pydot para hacer una traduccion de nuestro grafo a lenguage .dot
y guardamos el archivo en un archivo graphviz(.gv)
"""
import pydot
import os
import tempfile
from PIL import Image

def guardar_grafo(grafo, nombre_grafo=None):
    if grafo.es_dirigido():
        pydot_graph = pydot.Dot(graph_type="digraph")
    else:
        pydot_graph = pydot.Dot(graph_type="graph", strict=True)

    # Traduccion de los nodos
    for nodo in grafo.get_nodos().values():
        if nodo.get_distancia() != 0:
            node = pydot.Node(nodo.get_etiqueta(),
                              label="{} ({})".format(nodo.get_etiqueta(), str(nodo.get_distancia())))
        else:
            node = pydot.Node(nodo.get_etiqueta())
        pydot_graph.add_node(node)

    # Traduccion de las aristas
    for edge in grafo.get_aristas():
        etiqueta_nodo_fuente = edge.get_nodo_fuente().get_etiqueta()
        etiqueta_nodo_destino = edge.get_nodo_destino().get_etiqueta()
        peso = str(edge.get_peso())
        pydot_edge = pydot.Edge(etiqueta_nodo_fuente, etiqueta_nodo_destino)
        pydot_edge.set_label(peso)
        pydot_graph.add_edge(pydot_edge)

    pydot_graph.write_raw(nombre_grafo + ".gv")  # Escribimos el grafo en un archivo .gv
def mostrar_grafo(grafo):
    if grafo.es_dirigido():
        pydot_graph = pydot.Dot(graph_type="digraph")
    else:
        pydot_graph = pydot.Dot(graph_type="graph", strict=True)

    # Traduccion de los nodos
    for nodo in grafo.get_nodos().values():

        if nodo.get_distancia()!=0:
            node = pydot.Node(nodo.get_etiqueta(), label="{} ({})".format(nodo.get_etiqueta(),str(nodo.get_distancia())))
        else:
            node = pydot.Node(nodo.get_etiqueta())
        pydot_graph.add_node(node)

    # Traduccion de las aristas
    for edge in grafo.get_aristas():
        etiqueta_nodo_fuente = edge.get_nodo_fuente().get_etiqueta()
        etiqueta_nodo_destino = edge.get_nodo_destino().get_etiqueta()
        peso = str(edge.get_peso())
        pydot_edge = pydot.Edge(etiqueta_nodo_fuente, etiqueta_nodo_destino)
        pydot_edge.set_label(peso)
        pydot_graph.add_edge(pydot_edge)
    temp = tempfile.NamedTemporaryFile(delete=False)
    pydot_graph.write_png(temp.name)

    image = Image.open(temp.name)
    temp.close()

    image.show()
    os.unlink(temp.name)

# Generador de grafos e imágenes

In [6]:


# Generamos grafo de mallas con 30 nodos
grafo_malla_30 = Grafo(dirigido=False)
grafo_malla_30.grafo_malla(5, 6)
guardar_grafo(grafo_malla_30, "grafo_malla_30_nodos")
guardar_grafo(grafo_malla_30.KruskalD(),"grafo_malla_30_nodos_KruskalD")
guardar_grafo(grafo_malla_30.KruskalI(),"grafo_malla_30_nodos_KruskalI")
guardar_grafo(grafo_malla_30.Prim(),"grafo_malla_30_nodos_Prim")



# Generamos grafo de mallas con 500 nodos
grafo_malla_500 = Grafo(dirigido=False)
grafo_malla_500.grafo_malla(50, 10)
guardar_grafo(grafo_malla_500, "grafo_malla_500_nodos")
guardar_grafo(grafo_malla_500.KruskalD(), "grafo_malla_500_nodos_KruskalD")
guardar_grafo(grafo_malla_500.KruskalI(), "grafo_malla_500_nodos_KruskalI")
guardar_grafo(grafo_malla_500.Prim(), "grafo_malla_500_nodos_Prim")


# Generamos grafo Erdös y Rényi con 30 nodos y 200 aristas
grafo_erdos_30_200 = Grafo(dirigido=False)
grafo_erdos_30_200.grafo_erdos_renyi(30, 200)
guardar_grafo(grafo_erdos_30_200, "grafo_erdos_30_200")
guardar_grafo(grafo_erdos_30_200.KruskalD(), "grafo_erdos_30_200_KruskalD")
guardar_grafo(grafo_erdos_30_200.KruskalI(), "grafo_erdos_30_200_KruskalI")
guardar_grafo(grafo_erdos_30_200.Prim(), "grafo_erdos_30_200_Prim")


# Generamos grafo Erdös y Rényi con 500 nodos y 2500 aristas
grafo_erdos_500_2500 = Grafo(dirigido=False)
grafo_erdos_500_2500.grafo_erdos_renyi(500, 2500)
guardar_grafo(grafo_erdos_500_2500, "grafo_erdos_500_2500")
guardar_grafo(grafo_erdos_500_2500.KruskalD(),"grafo_erdos_500_2500_KruskalD")
guardar_grafo(grafo_erdos_500_2500.KruskalI(),"grafo_erdos_500_2500_KruskalI")
guardar_grafo(grafo_erdos_500_2500.Prim(),"grafo_erdos_500_2500_Prim")



# generamos grafo con modelo de Gilbert 30 nodos y probabilidad 0.5
grafo_gilbert_30_05 = Grafo(dirigido=False)
grafo_gilbert_30_05.grafo_gilbert(30, 0.5)
guardar_grafo(grafo_gilbert_30_05, "grafo_gilbert_30_05")
guardar_grafo(grafo_gilbert_30_05.KruskalD(),"grafo_gilbert_30_05_KruskalD")
guardar_grafo(grafo_gilbert_30_05.KruskalI(),"grafo_gilbert_30_05_KruskalI")
guardar_grafo(grafo_gilbert_30_05.Prim(),"grafo_gilbert_30_05_Prim")


# generamos grafo con modelo de Gilbert 500 nodos y probabilidad 0.02
grafo_gilbert_500_002 = Grafo(dirigido=False)
grafo_gilbert_500_002.grafo_gilbert(500, 0.02)
guardar_grafo(grafo_gilbert_500_002, "grafo_gilbert_500_002")
guardar_grafo(grafo_gilbert_500_002.KruskalD(),"grafo_gilbert_500_002_KruskalD()")
guardar_grafo(grafo_gilbert_500_002.KruskalI(),"grafo_gilbert_500_002_KruskalI()")
guardar_grafo(grafo_gilbert_500_002.Prim(),"grafo_gilbert_500_002_Prim")


# generamos grafo con modelo geográfico simple con 30 nodos y r=0.5
grafo_geografico_30_05 = Grafo(dirigido=False)
grafo_geografico_30_05.grafo_geografico(30, 0.5)
guardar_grafo(grafo_geografico_30_05, "grafo_geografico_30_05")
guardar_grafo(grafo_geografico_30_05.KruskalD(), "grafo_geografico_30_05_KruskalD")
guardar_grafo(grafo_geografico_30_05.KruskalI(), "grafo_geografico_30_05_KruskalI")
guardar_grafo(grafo_geografico_30_05.Prim(), "grafo_geografico_30_05_Prim")


# generamos grafo con modelo geográfico simple con 500 nodos y r=0.1
grafo_geografico_500_01 = Grafo(dirigido=False)
grafo_geografico_500_01.grafo_geografico(500, 0.15)
guardar_grafo(grafo_geografico_500_01, "grafo_geografico_500_01")
guardar_grafo(grafo_geografico_500_01.KruskalD(), "grafo_geografico_500_01_KruskalD")
guardar_grafo(grafo_geografico_500_01.KruskalI(), "grafo_geografico_500_01_KruskalI")
guardar_grafo(grafo_geografico_500_01.Prim(), "grafo_geografico_500_01_Prim")


# generamos grafo con modelo Barabási-Albert con 30 nodos y grado 10
grafo_babarasi_30_10 = Grafo(dirigido=False)
grafo_babarasi_30_10.grafo_barabasi_albert(30, 10, False, False)
guardar_grafo(grafo_babarasi_30_10, "grafo_babarasi_30_10")
guardar_grafo(grafo_babarasi_30_10.KruskalD(), "grafo_babarasi_30_10_Kruskal_D")
guardar_grafo(grafo_babarasi_30_10.KruskalI(), "grafo_babarasi_30_10_Kruskal_I")
guardar_grafo(grafo_babarasi_30_10.Prim(), "grafo_babarasi_30_10_Prim")


# generamos grafo con modelo Barabási-Albert con 500 nodos y grado 12
grafo_babarasi_500_12 = Grafo(dirigido=False)
grafo_babarasi_500_12.grafo_barabasi_albert(500, 15)
guardar_grafo(grafo_babarasi_500_12, "grafo_babarasi_500_12")
guardar_grafo(grafo_babarasi_500_12.KruskalD(), "grafo_babarasi_500_12_KruskalD")
guardar_grafo(grafo_babarasi_500_12.KruskalI(), "grafo_babarasi_500_12_KruskalI")
guardar_grafo(grafo_babarasi_500_12.Prim(), "grafo_babarasi_500_12_Prim")

# generamos grafo con modelo Dorogovtsev-Mendes con 30 nodos
grafo_dorogovtsev_mendes_30 = Grafo(dirigido=False)
grafo_dorogovtsev_mendes_30.dorogovtsev_mendes(30)
guardar_grafo(grafo_dorogovtsev_mendes_30, "grafo_dorogovtsev_mendes_30")
guardar_grafo(grafo_dorogovtsev_mendes_30.KruskalD(), "grafo_dorogovtsev_mendes_30_KruskalD")
guardar_grafo(grafo_dorogovtsev_mendes_30.KruskalI(), "grafo_dorogovtsev_mendes_30_KruskalI")
guardar_grafo(grafo_dorogovtsev_mendes_30.Prim(), "grafo_dorogovtsev_mendes_30_Prim")


# generamos grafo con modelo Dorogovtsev-Mendes con 500 nodos
grafo_dorogovtsev_mendes_500 = Grafo(dirigido=False)
grafo_dorogovtsev_mendes_500.dorogovtsev_mendes(500)
guardar_grafo(grafo_dorogovtsev_mendes_500, "grafo_dorogovtsev_mendes_500")
guardar_grafo(grafo_dorogovtsev_mendes_500.KruskalD(), "grafo_dorogovtsev_mendes_500_KruskalD")
guardar_grafo(grafo_dorogovtsev_mendes_500.KruskalI(), "grafo_dorogovtsev_mendes_500_KruskalI")
guardar_grafo(grafo_dorogovtsev_mendes_500.Prim(), "grafo_dorogovtsev_mendes_500_Prim")






Se truncaron las últimas líneas 5000 del resultado de transmisión.
346 - 31 - 437 
178 - 32 - 351 
71 - 32 - 256 
98 - 32 - 468 
131 - 4 - 316 
99 - 33 - 313 
180 - 17 - 462 
359 - 6 - 361 
17 - 53 - 442 
234 - 34 - 499 
31 - 52 - 419 
301 - 6 - 358 
184 - 46 - 398 
178 - 28 - 363 
236 - 62 - 384 
151 - 45 - 238 
254 - 29 - 495 
38 - 9 - 39 
67 - 43 - 103 
184 - 20 - 270 
315 - 59 - 340 
188 - 12 - 312 
229 - 18 - 380 
163 - 29 - 267 
324 - 44 - 373 
295 - 24 - 309 
103 - 47 - 400 
163 - 5 - 396 
221 - 5 - 290 
22 - 6 - 319 
239 - 23 - 353 
157 - 35 - 481 
134 - 34 - 232 
304 - 51 - 352 
26 - 8 - 207 
109 - 130 - 323 
80 - 52 - 391 
360 - 62 - 487 
18 - 37 - 381 
75 - 38 - 309 
115 - 26 - 153 
200 - 46 - 311 
96 - 49 - 410 
491 - 53 - 497 
195 - 109 - 445 
255 - 57 - 403 
136 - 59 - 288 
324 - 11 - 371 
172 - 118 - 385 
108 - 19 - 147 
206 - 43 - 224 
250 - 42 - 294 
111 - 31 - 322 
174 - 25 - 486 
82 - 69 - 234 
68 - 4 - 188 
38 - 38 - 59 
35 - 60 - 348 
49 - 103 - 168 
239 - 20 - 458